# Reprezentacja sekwencji szkieletowych

- Normalizacja pozycji:
    - np. odjęcie pozycji biodra jako punktu odniesienia (położenie względne),
    - normalizacja skali.
- Radzenie sobie z różną długością sekwencji:
    - przycięcie / uzupełnianie (padding) do ustalonej długości,
    - ewentualnie próbkowanie co N klatek.

Punktem odniesienia został ustawiony punkt pozycji biodra. W COCO punkty biodra są oznaczone indeksami 11 (lewe biodro) oraz 12 (prawe biodro), co jest napisane w wygenerowanych plikach JSON po detekcji.

```json
"meta_info": {
		"dataset_name": "coco",
		"num_keypoints": 17,
		"keypoint_id2name": {
			"0": "nose",
			"1": "left_eye",
			"2": "right_eye",
			"3": "left_ear",
			"4": "right_ear",
			"5": "left_shoulder",
			"6": "right_shoulder",
			"7": "left_elbow",
			"8": "right_elbow",
			"9": "left_wrist",  
			"10": "right_wrist",
			"11": "left_hip",  // <-- lewe biodro
			"12": "right_hip", // <-- prawe biodro
			"13": "left_knee",
			"14": "right_knee",
			"15": "left_ankle",
			"16": "right_ankle"
		},
}
```

Szkielet postaci w detektorze zawiera **17 punktów kluczowych (keypointów)**, które są oznaczone współrzędnymi **[x,y]**. Stąd późniejszy rozmiar (17, 2) np:

```json
"keypoints": [
		[
		205.00148010253906, // <-- X
		322.10467529296875  // <-- Y
		],
		itd...
]
```

In [1]:
import os
import glob
import json
import numpy as np
import random
import pandas as pd
from tqdm import tqdm

Funkcja `normalize_frame` przyjmuje tablice punktów (17, 2) dla jednej klatki.

Zwraca znormalizowane punkty względem środka bioder (średnia arytmetyczna z lewego i prawego biodra).

In [2]:
def normalize_frame(keypoints):
    kps = np.array(keypoints)
    
    # pozycje bioder
    left_hip = 11
    right_hip = 12

    # pozycja środka bioder
    hip_center = (kps[left_hip] + kps[right_hip]) / 2.0

    # centrownie punktów względem bioder
    centered_kps = kps - hip_center

    # skalowanie - wybieramy najwyżej i najniżej położone keypointy na osi Y jako odniesienie do wyznaczenia wysokości
    min_y = np.min(centered_kps[:, 1])
    max_y = np.max(centered_kps[:, 1])
    height = max_y - min_y + 1e-6 # dodaje bardzo małą wartość aby uniknąć dzielenia przez 0

    normalized_kps = centered_kps / height

    return normalized_kps

Funkcja `load_data` wczytuje sekwencje ...

In [21]:
def load_data(root_dir, fall_csv, adl_csv):
    cols = ['sequence_name', 'frame_number', 'label']
    fall = pd.read_csv(fall_csv, header=None, usecols=[0, 1, 2], names=cols)
    adl = pd.read_csv(adl_csv, header=None, usecols=[0, 1, 2], names=cols)
    df = pd.concat([fall, adl], ignore_index=True)

    keypoints = []
    labels = []

    # szukanie wszystkich plików json
    search_pattern = os.path.join(root_dir, "*", "*.json")
    json_files = glob.glob(search_pattern)
    print(f"Znaleziono {len(json_files)} plików JSON.")

    # --- pętla przetwarzająca ---
    for json_path in tqdm(json_files, desc="Przetwarzanie plików"):

        filename = os.path.basename(json_path)

    # --- 
        seq_name = None
        if "fall-" in filename:
            idx = filename.find("fall-")
            seq_name = filename[idx : idx+7]
        elif 'adl-' in filename:
            idx = filename.find("adl-")
            seq_name = filename[idx : idx+6]

        if not seq_name:
            continue

        seq_df = df[df['sequence_name'] == seq_name]

        if seq_df.empty:
            continue

    # --- wczytanie jsona ---
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"Błąd odczytu {json_path}: {e}")
            continue

    # --- ekstrakcja klatek ---
        if 'instance_info' not in data:
            print(f"Brak pola 'instace_info' w {json_path}")
            continue
        
        # sortowanie klatek dla pewności
        # frames_data = sorted(data['instance_info'], key=lambda x: x['frame_id'])

        for frame in data['instance_info']:
            
            frame_id = frame['frame_id']

        # --- dopasowanie etykiety ---
            row = seq_df[seq_df['frame_number'] == frame_id]
            if row.empty:
                continue

            raw_label = row.iloc[0]['label']

            # 0 to 'fall' więc pomijamy zgodnie z instrukcją
            if raw_label == 0:
                continue
            
            # mapowanie na klasy 0/1
            # jeżeli -1 (postać nie leży) => 0
            # jeżeli 1 (postać leży) => 1
            binary_label = 0 if raw_label == -1 else 1 

        # --- ekstrakcja punktów ---
            if not frame.get("instances"):
                continue

            raw_kps = frame['instances'][0]['keypoints'] # zakładamy jedną osobę

            # normalizacja względem bioder
            norm_kps = normalize_frame(raw_kps)

            keypoints.append(norm_kps)
            labels.append(binary_label)

    return np.array(keypoints), np.array(labels)

Uruchomienie + raport

In [22]:
# ścieżka do głównego folderu z wynikami po detekcji
csv_file_fall = "../UR_fall_det/csv/urfall-cam0-falls.csv"
csv_file_adl = "../UR_fall_det/csv/urfall-cam0-adls.csv"
results_folder = "../results"

X, y = load_data(results_folder, csv_file_fall, csv_file_adl)

print("\n--- RAPORT ---")
print(f"Wczytano {len(X)} sekwencji")

Znaleziono 69 plików JSON.


Przetwarzanie plików: 100%|██████████| 69/69 [00:06<00:00, 10.37it/s]


--- RAPORT ---
Wczytano 9627 sekwencji


Przykładowa sekwencja

> X[X] - klatka **X** wszystkie keypointy

> X[X][Y] - keypointy w stawie **Y** w klatce **X**

> X[X][Y][Z] - keypoint(x,y) **Z** z keypointów **Y** w klatce **X**

In [60]:
rand = random.randint(0, len(X) - 1)
print(f"Kształt danych: {X.shape} (Klatki, Punkty, XY)")

print("\nPrzykładowe znormalizowane punkty (keypointy):")
print("* punkty 11 i 12 powinny być blisko 0 bo to punkty biodra")
print("          X           Y")
for i in range(17):
    print(f"{i}: {X[rand][i]} {('*' if i==11 or i==12 else '')}")

print("\nLosowe 10 klatek z etykietami (0 - stoi, 1 - leży)")
for _ in range(0, 10):
    rand = random.randint(0, len(X) - 1)
    print(f'Klatka[{rand}] | {y[rand]}')

Kształt danych: (9627, 17, 2) (Klatki, Punkty, XY)

Przykładowe znormalizowane punkty (keypointy):
* punkty 11 i 12 powinny być blisko 0 bo to punkty biodra
          X           Y
0: [-0.14285711 -0.39880952] 
1: [-0.13095234 -0.43452377] 
2: [-0.15476188 -0.42261903] 
3: [-0.05952384 -0.47023808] 
4: [-0.16666665 -0.42261903] 
5: [ 0.08333326 -0.37499998] 
6: [-0.11904769 -0.30357143] 
7: [ 0.14285711 -0.20833334] 
8: [-0.13095234 -0.14880949] 
9: [ 0.14285711 -0.04166663] 
10: [-0.16666665  0.00595238] 
11: [ 0.05952384 -0.00595238] *
12: [-0.05952384  0.00595238] *
13: [0.03571431 0.23214282] 
14: [-0.03571431  0.25595235] 
15: [0.1071428  0.48214284] 
16: [0.         0.52976192] 

Losowe 10 klatek z etykietami (0 - stoi, 1 - leży)
Klatka[2021] | 1
Klatka[5623] | 1
Klatka[903] | 0
Klatka[5721] | 0
Klatka[2442] | 0
Klatka[4242] | 0
Klatka[6407] | 0
Klatka[8122] | 0
Klatka[1575] | 0
Klatka[9545] | 0


In [62]:
print("Zapisywanie datasetu...")
np.savez_compressed('../data/dataset_urfall.npz', X=X, y=y)
print("Gotowe! Plik 'dataset_urfall.npz' został utworzony w folderze ../data.")

Zapisywanie datasetu...
Gotowe! Plik 'dataset_urfall.npz' został utworzony w folderze ../data.
